# APIM ❤️ OpenAI

## バックエンドサーキットブレーカラボ
![flow](../../images/backend-circuit-breaking.gif)

Azure OpenAIエンドポイントまたはモックサーバーに対して、APIMの組み込み[バックエンドサーキットブレーカ機能](https://learn.microsoft.com/en-us/azure/api-management/backends?tabs=bicep)を試すためのプレイグラウンド。

### 前提条件
- [Python 3.8以降のバージョン](https://www.python.org/)がインストールされていること
- [VS Code](https://code.visualstudio.com/)がインストールされ、[Jupyterノートブック拡張機能](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)が有効になっていること
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)がインストールされていること
- Contributor権限を持つ[Azureサブスクリプション](https://azure.microsoft.com/en-us/free/)があること
- [Azure OpenAIへのアクセス](https://aka.ms/oai/access)が許可されていること、またはモックサービスを有効にすること
- [Azure CLIでAzureにサインイン](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)していること

### 0️⃣ ノートブック変数の初期化
このラボを実際のAzure OpenAIエンドポイントで使用する場合は、```mock_disabled``` 変数を ```True``` に設定し、モックサーバーで同等の動作をシミュレートする場合は ```False``` に設定します。
- ```mock_webapps``` 変数は、モック機能のためにデプロイされたWebアプリのリストを設定します。
- ロケーションパラメータは、[Azureリージョンごとの製品の利用可能性](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)に基づいて調整してください。
- OpenAIモデルとバージョンは、[リージョンごとの利用可能性](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)に基づいて調整してください。

In [ ]:
import os
import json
import datetime
import requests
notebook_path = os.path.abspath("")
notebook_name = os.path.basename(globals()['__vsc_ipynb_file__'])

lab_prefix = "av4" # used to ensure unique names within Azure
mock_disabled = True
mock_webapps = [{"name": "openaimock1"}] # ensure that the names are not being used within Azure
resource_group = "lab-ai-gateway"
apim_resource_name = lab_prefix + "-aigw-apim"
apim_resource_location = "eastus"
apim_resource_sku = "Consumption"
openai_resources = [ {"name": lab_prefix + "-aigw-openai1", "location": "eastus"}]
openai_resource_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'

### 1️⃣ Azureリソースグループの作成
このラボでデプロイされるすべてのリソースは、指定されたリソースグループに作成されます。

In [ ]:
resource_group_stdout = ! az group create --name {resource_group} --location {apim_resource_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group, " created ⌚ ", datetime.datetime.now().time())

### 2️⃣ Azure OpenAIリソースの作成
Azure OpenAIサービスは、OpenAIの強力な言語モデルへのREST APIアクセスを提供します。以下のスクリプトは[このクイックスタート](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=cli)に基づいており、新しいAzure OpenAIリソースを作成します。
- 注意: 既存のリソースを再利用したい場合は、このステップをスキップしてください。

In [ ]:
if mock_disabled:
    openai_resource_name = openai_resources[0].get("name")
    openai_resource_location = openai_resources[0].get("location")
    openai_resource_stdout = ! az cognitiveservices account create --name {openai_resource_name} --resource-group {resource_group} \
                --kind OpenAI --sku-name {openai_resource_sku} --location {openai_resource_location} --custom-domain {openai_resource_name}
    if openai_resource_stdout.n.startswith("ERROR"):
        print(openai_resource_stdout)
    else:
        print("✅ Azure OpenAI resource created ⌚ ", datetime.datetime.now().time())
else:
    print("🚧 Mock enabled, skipping Azure OpenAI resource creation")

### 3️⃣ モデルのデプロイ
Azure OpenAIリソースを作成したら、APIコールを開始する前にモデルをデプロイする必要があります。以下のスクリプトは、指定されたデプロイ名、モデル名、およびモデルバージョンを使用してモデルデプロイメントを作成します。

In [ ]:
if mock_disabled:
    openai_resource_name = openai_resources[0].get("name")
    openai_deployment_stdout = ! az cognitiveservices account deployment create --name {openai_resource_name} --resource-group  {resource_group} \
        --deployment-name {openai_deployment_name} --model-name {openai_model_name} --model-version {openai_model_version}  --model-format OpenAI 
    if openai_deployment_stdout.n.startswith("ERROR"):
        print(openai_deployment_stdout)
    else:
        print("✅ OpenAI deployment created ⌚ ", datetime.datetime.now().time())
else:
    print("🚧 Mock enabled, skipping OpenAI deployment creation")

### 4️⃣ API Management (APIM) リソースの作成
APIMはOpenAI APIのAIゲートウェイとして機能します。以下のスクリプトは[このクイックスタート](https://learn.microsoft.com/en-us/azure/api-management/get-started-create-service-instance-cli)に基づいています。
- 注意: 既存のインスタンスを再利用したい場合は、このステップをスキップしてください。

In [ ]:
apim_resource_stdout = ! az apim create -g {resource_group} -n {apim_resource_name} -l {apim_resource_location} \
    --sku-name {apim_resource_sku} --publisher-email noreply@microsoft.com --publisher-name Microsoft --enable-managed-identity
if apim_resource_stdout.n.startswith("ERROR"):
    print(apim_resource_stdout)
else:
    print("✅ Azure API Management resource created ⌚ ", datetime.datetime.now().time())


### 5️⃣ API Management (APIM) リソースの詳細を取得
APIMインスタンスは、[管理されたAPIゲートウェイ](https://learn.microsoft.com/en-us/azure/api-management/api-management-gateways-overview)、[システム管理アイデンティティ](https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-use-managed-service-identity)、およびマスター[サブスクリプションキー](https://learn.microsoft.com/en-us/azure/api-management/api-management-subscriptions)を提供します。このラボではマスターサブスクリプションキーを使用しますが、本番環境ではAPI消費者のために新しいサブスクリプションキーを作成する必要があります。

In [ ]:
apim_resource_stdout = ! az apim show -g {resource_group} -n {apim_resource_name}
apim_resource = json.loads(apim_resource_stdout.n)
apim_resource_id = apim_resource.get("id")
apim_resource_gateway_url = apim_resource.get("gatewayUrl")
apim_managed_identity = apim_resource.get("identity").get("principalId")
apim_subscription_key = ! az rest --method POST --uri {apim_resource_id}/subscriptions/master/listSecrets?api-version=2022-08-01 --query primaryKey -o tsv
apim_subscription_key = apim_subscription_key.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

### 6️⃣ APIMがOpenAI APIにアクセスできるようにロールを割り当てる
このラボでは、[Azureマネージドアイデンティティ](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/managed-identity)を使用したキーなしアプローチのゼロトラストセキュリティ戦略を採用しています。以下のスクリプトは、APIMのマネージドアイデンティティに「Cognitive Services OpenAI User」ロールを割り当て、OpenAI APIにアクセスできるようにします。

In [ ]:
if mock_disabled:
    openai_resource_name = openai_resources[0].get("name")
    openai_resource_stdout = ! az cognitiveservices account show --name {openai_resource_name} --resource-group {resource_group}
    openai_resource = json.loads(openai_resource_stdout.n)
    openai_resource_id = openai_resource.get("id")
    role_assignment_stdout = ! az role assignment create --assignee {apim_managed_identity} \
        --role "Cognitive Services OpenAI User" \
        --scope {openai_resource_id}
    if role_assignment_stdout.n.startswith("ERROR"):
        print(role_assignment_stdout)
    else:
        print("✅ Role assignment created ⌚ ", datetime.datetime.now().time())
else:
    print("🚧 Mock enabled, skipping Role assignment")

### 7️⃣ サーキットブレーカ構成を持つAPIMバックエンドの作成
サーキットブレーカ機能はプレビュー段階にあるため、このスクリプトはGA（一般提供）時に更新される可能性があります。詳細については[このドキュメント](https://learn.microsoft.com/en-us/azure/api-management/backends?tabs=bicep)を確認してください。
- 注意: ```interval``` および ```tripDuration``` パラメータはISO Timespan形式である必要があります。以下の例では、5分間の間に3つ以上の429ステータスコードが発生するとサーキットブレーカが作動します。サーキットブレーカは1分後にリセットされます。

In [ ]:
if mock_disabled:
    openai_resource_name = openai_resources[0].get("name")
    openai_resource_stdout = ! az cognitiveservices account show --name {openai_resource_name} --resource-group {resource_group}
    openai_resource = json.loads(openai_resource_stdout.n)
    openai_resource_endpoint = openai_resource.get("properties").get("endpoint")
    print("👉🏻 Azure OpenAI endpoint: ", openai_resource_endpoint)    
    backend_properties = {
        "properties": {
            "title": openai_resource_name + " backend",
            "url": openai_resource_endpoint + "/openai",
            "description": "Backend for OpenAI resource " + openai_resource_name,
            "protocol": "http",
            "circuitBreaker": {
                "rules": [
                    {
                        "failureCondition": {
                            "count": 3,
                            "errorReasons": [
                                "Server errors"
                            ],
                            "interval": "PT5M",
                            "statusCodeRanges": [
                                {
                                "min": 429,
                                "max": 429
                                }
                            ]
                        },
                        "name": "myBreakerRule",
                        "tripDuration": "PT1M"
                    }
                ]
            }
        }
    }
    uri = apim_resource_id + "/backends/" + openai_resource_name + "?api-version=2023-05-01-preview"
    backend_properties_text = "\"" + json.dumps(backend_properties).replace("\"","\\\"") + "\""
    backend_creation_stdout = ! az rest --method PUT --uri {uri} --body {backend_properties_text}
    if backend_creation_stdout.n.startswith("ERROR"):
        print(backend_creation_stdout)
    else:
        print("✅ Backend ", openai_resource_name," created ⌚ ", datetime.datetime.now().time())    
else:
    mock_webapp_name = mock_webapps[0].get("name")
    openai_resource_endpoint = "https://" + mock_webapp_name + ".azurewebsites.net"
    print("🚧 Mock enabled, using Mock endpoint instead of Azure OpenAI endpoint: ", openai_resource_endpoint)
    backend_properties = {
        "properties": {
            "title": mock_webapp_name + " backend",
            "url": openai_resource_endpoint + "/openai",
            "description": "Backend for Mock server " + mock_webapp_name,
            "protocol": "http",
            "circuitBreaker": {
                "rules": [
                    {
                        "failureCondition": {
                            "count": 3,
                            "errorReasons": [
                                "Server errors"
                            ],
                            "interval": "PT5M",
                            "statusCodeRanges": [
                                {
                                "min": 429,
                                "max": 429
                                }
                            ]
                        },
                        "name": "myBreakerRule",
                        "tripDuration": "PT1M"
                    }
                ]
            }
        }
    }
    uri = apim_resource_id + "/backends/" + mock_webapp_name + "?api-version=2023-05-01-preview"
    backend_properties_text = "\"" + json.dumps(backend_properties).replace("\"","\\\"") + "\""
    backend_creation_stdout = ! az rest --method PUT --uri {uri} --body {backend_properties_text}
    if backend_creation_stdout.n.startswith("ERROR"):
        print(backend_creation_stdout)
    else:
        print("✅ Backend ", mock_webapp_name," created ⌚ ", datetime.datetime.now().time())    


### 8️⃣ OpenAI APIをAPIMにインポートする
以下のスクリプトは、[公開されている](https://github.com/Azure/azure-rest-api-specs/tree/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable)json OpenAPI仕様を使用してOpenAI推論APIをインポートします。サブスクリプションキーのヘッダー名は、OpenAI APIで使用されるのと同じ名前である```api-key```に設定されます。

In [ ]:
if mock_disabled:
    openai_resource_stdout = ! az cognitiveservices account show --name {openai_resource_name} --resource-group {resource_group}
    openai_resource = json.loads(openai_resource_stdout.n)
    openai_resource_endpoint = openai_resource.get("properties").get("endpoint")
    print("👉🏻 OpenAI endpoint: ", openai_resource_endpoint)    
else:
    openai_resource_endpoint = "https://" + mock_webapps[0].get("name") + ".azurewebsites.net" # this lab dosn't implement load balancing
    print("🚧 Mock enabled, using Mock endpoint instead of Azure OpenAI endpoint: ", openai_resource_endpoint)
apim_api_import_stdout = ! az apim api import --resource-group {resource_group} --service-name {apim_resource_name} \
        --api-id "openai" --path "openai" --api-type "http" --display-name "OpenAI" --description "OpenAI inference API" \
        --service-url {openai_resource_endpoint}"/openai" --protocols "https" \
        --specification-format OpenApiJson --specification-url {openai_specification_url} \
        --subscription-required true --subscription-key-header-name "api-key" --subscription-key-query-param-name "api-key"
if apim_api_import_stdout.n.startswith("ERROR"):
    print(apim_api_import_stdout)
else:
    print("✅ API imported on ", datetime.datetime.now().time())

### 8️⃣ APIポリシーを更新して自己管理アイデンティティを取得し、Azure OpenAIに認証するためのベアラートークンを送信する
APIポリシーには、割り当てられたマネージドアイデンティティを使用してAzure OpenAI APIへのリクエストを認証するための[ドキュメント化されたポリシースニペット](https://learn.microsoft.com/en-us/azure/api-management/api-management-authenticate-authorize-azure-openai#authenticate-with-managed-identity)を含める必要があります。
- 注意: Azure CLIを通じてポリシーを追加する機能は[まだ利用できません](https://github.com/Azure/azure-cli/issues/14695)。そのため、代わりに```az rest```コマンドを使用しています。

In [ ]:
if mock_disabled:
    backend_id = openai_resources[0].get("name")
else:
    backend_id = mock_webapps[0].get("name")
with open(notebook_path + "/policy.xml", 'r') as policy_xml_file:
    policy_xml = policy_xml_file.read()
    policy_xml = policy_xml.replace("{backend-id}", backend_id)
with open(notebook_name.replace('ipynb','json'), 'w') as policy_json_file:
    policy_json_file.write("{\"properties\":{\"value\":\"" + policy_xml.replace("\"","\\\"") + "\"} }")
uri = apim_resource_id + "/apis/openai/policies/policy?api-version=2022-09-01-preview"
body_file_path = "@" + notebook_path + "/" + notebook_name.replace('ipynb','json')
apim_policy_stdout = ! az rest --method PUT --uri {uri} --body {body_file_path}
os.remove(notebook_name.replace('ipynb','json'))
print("✅ Policy updated ⌚ ", datetime.datetime.now().time())

### 🧪 直接HTTPコールを使用したAPIのテスト
Requestsは、ここで生のAPIリクエストを行い、レスポンスを検査するために使用される、エレガントでシンプルなPython用のHTTPライブラリです。

In [ ]:
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version
if mock_disabled:
    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}
else:
    messages={
        "messages": [
            {
                "role": "system", 
                "content": {
                    "simulation": {
                        "default": {"response_status_code": 200, "wait_time_ms": 0},
                        "openaimock1.azurewebsites.net": {"response_status_code": 429}
                    }
                }
            }
        ]
    }
response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)
print("status code: ", response.status_code)
print("headers ", response.headers)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("response: ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)

### 🧪 Azure OpenAI Python SDKを使用したAPIのテスト
OpenAPIは広く使用されている[Pythonライブラリ](https://github.com/openai/openai-python)を提供しています。このライブラリには、すべてのリクエストパラメータとレスポンスフィールドの型定義が含まれています。このテストの目的は、APIMがOpenAIへのリクエストをシームレスにプロキシし、その機能を妨げることなく動作することを確認することです。
- 注意: このステップを実行する前に、ターミナルで```pip install openai```を実行してください。

In [ ]:
from openai import AzureOpenAI
if mock_disabled:
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]
else:
    messages=[
            {
                "role": "system", 
                "content": {
                    "simulation": {
                        "default": {"response_status_code": 200, "wait_time_ms": 0}
                    }
                }
            }
        ]
client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription_key,
    api_version=openai_api_version
)
response = client.chat.completions.create(model=openai_model_name, messages=messages)
print(response.choices[0].message.content)

### 🗑️ リソースのクリーンアップ

ラボが終了したら、追加の料金を避け、Azureサブスクリプションを整理するために、デプロイしたすべてのリソースをAzureから削除する必要があります。リソースグループを削除するのが、作成したすべてのAzureリソースを削除する最も速い方法です。

In [ ]:
run_cell = True
if run_cell:
    ! az group delete --name {resource_group} -y
    ! az apim deletedservice purge --service-name {apim_resource_name} --location {apim_resource_location}
    if mock_disabled:
        openai_resource_name = openai_resources[0].get("name")
        ! az cognitiveservices account purge -g {resource_group} -n {openai_resource_name} -l {openai_resource_location}